In [10]:
# Import the class from the Python file (module)
import pandas as pd
import matplotlib.pyplot as plt
import os
# from dotenv import load_dotenv
# from pathlib import Path
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from BinanceClient import BinanceClient
import numpy as np
from typing import Final
import joblib
from BatchFeatures import BatchFeatures
from datetime import datetime, timedelta
%matplotlib widget

## Load pair df

In [11]:
import os
from datetime import datetime, timedelta, timezone

def interval_slug(s: str) -> str:
    return s.strip().replace(" ", "").replace("/", "").lower()

def make_db_name(pair: str, interval: str, weeks: int) -> str:
    return f"{pair}_{interval_slug(interval)}_{weeks}weeks.db"

def load_or_fetch_pair_df(pair: str, interval: str, weeks: int) -> tuple[str, "pd.DataFrame"]:
    db_name = make_db_name(pair, interval, weeks)
    db_path = db_name

    print(f"[{pair}] DB: {db_path}")

    binance_client = BinanceClient(db_path)
    binance_client.set_interval(interval)

    df = None

    if os.path.exists(db_path):
        df = binance_client.fetch_data_from_db(pair)
        if df is not None and not df.empty:
            print(f"[{pair}] Loaded {len(df):,} rows from DB.")
        else:
            df = None

    if df is None:
        print(f"[{pair}] No usable DB data found -> fetching from Binance...")

        api_secret = os.getenv("BINANCE_SECRET_KEY")
        api_key = os.getenv("BINANCE_API_KEY")
        binance_client.make(api_key, api_secret)

        server_time = binance_client.get_server_time()
        end_dt = datetime.fromtimestamp(server_time["serverTime"] / 1000, tz=timezone.utc)
        start_dt = end_dt - timedelta(weeks=weeks)

        start_ms = int(start_dt.timestamp() * 1000)
        end_ms = int(end_dt.timestamp() * 1000)

        data = binance_client.fetch_data(pair, start_ms, end_ms)
        if data is None or data.empty:
            raise RuntimeError(f"[{pair}] No data returned from Binance for the requested range.")

        binance_client.store_data_to_db(pair, data)

        df = binance_client.fetch_data_from_db(pair)
        if df is None or df.empty:
            raise RuntimeError(f"[{pair}] Data fetched/stored but DB load returned empty.")

        print(f"[{pair}] Fetched + stored + loaded {len(df):,} rows.")

    df = df.sort_index()
    return db_path, df


## Load BTC + ETH, then align timestamps

In [27]:
import pandas as pd

interval = "5m"
weeks = 52
pairs = ["BTCUSDT", "ETHUSDT"]

paths = {}
dfs = {}

for p in pairs:
    db_path, df = load_or_fetch_pair_df(p, interval, weeks)
    paths[p] = db_path
    dfs[p] = df

btc = dfs["BTCUSDT"].copy()
eth = dfs["ETHUSDT"].copy()

# Ensure index is datetime and UTC-consistent (should already be)
btc.index = pd.to_datetime(btc.index, utc=False)
eth.index = pd.to_datetime(eth.index, utc=False)

# Inner-join on timestamp intersection (avoid forward-fill unless you really want it)
joined = (
    btc.add_prefix("btc_")
       .join(eth.add_prefix("eth_"), how="inner")
       .sort_index()
)

def add_eth_trend(df, ma_bars=12*24*7):  # ~1 week MA on 5m
    out = df.copy()
    out["eth_ma"] = out["eth_close"].rolling(ma_bars).mean()
    out["eth_trend_ok"] = out["eth_close"] > out["eth_ma"]
    return out

joined3 = add_eth_trend(joined2).dropna()


print("Joined rows:", len(joined))
print("Start:", joined.index.min(), "End:", joined.index.max())
print(joined.head())


[BTCUSDT] DB: BTCUSDT_5m_52weeks.db
[BTCUSDT] Loaded 104,832 rows from DB.
[ETHUSDT] DB: ETHUSDT_5m_52weeks.db
[ETHUSDT] Loaded 104,832 rows from DB.
Joined rows: 104818
Start: 2025-01-25 16:30:00 End: 2026-01-24 15:15:00
                      btc_open   btc_high    btc_low  btc_close  btc_volume  \
timestamp                                                                     
2025-01-25 16:30:00  104786.49  104791.33  104679.35  104719.99    36.23060   
2025-01-25 16:35:00  104719.99  104815.99  104719.99  104749.46    58.02236   
2025-01-25 16:40:00  104749.46  104765.49  104599.39  104638.00    58.29234   
2025-01-25 16:45:00  104637.99  104741.76  104591.67  104671.21    48.11209   
2025-01-25 16:50:00  104671.21  104717.90  104659.52  104688.50    27.81774   

                     eth_open  eth_high  eth_low  eth_close  eth_volume  
timestamp                                                                
2025-01-25 16:30:00   3343.44   3344.95  3335.69    3339.98    906.9382  
20

## Build relative-value “spread” features

In [13]:
import numpy as np

def add_rv_features(df: pd.DataFrame, beta_window=12*24*7, z_window=12*24*7):
    # 1 week defaults on 5m: 12 bars/hr * 24 * 7 = 2016
    out = df.copy()

    out["log_btc"] = np.log(out["btc_close"])
    out["log_eth"] = np.log(out["eth_close"])

    # Rolling beta: regress log_eth on log_btc (simple rolling OLS via cov/var)
    x = out["log_btc"]
    y = out["log_eth"]
    cov = y.rolling(beta_window).cov(x)
    var = x.rolling(beta_window).var()
    out["beta"] = cov / var

    out["spread"] = out["log_eth"] - out["beta"] * out["log_btc"]

    mu = out["spread"].rolling(z_window).mean()
    sd = out["spread"].rolling(z_window).std(ddof=0)
    out["z"] = (out["spread"] - mu) / sd

    # optional helpers
    out["z_change"] = out["z"].diff()
    out["corr"] = out["log_eth"].rolling(z_window).corr(out["log_btc"])

    return out

joined2 = add_rv_features(joined)
joined2 = joined2.dropna()  # drop warmup region
print(joined2[["beta","spread","z","corr"]].tail())


                         beta     spread         z      corr
timestamp                                                   
2026-01-24 14:55:00  1.874402 -13.378103 -1.968378  0.986454
2026-01-24 15:00:00  1.874884 -13.383005 -1.967502  0.986460
2026-01-24 15:05:00  1.875392 -13.387315 -1.966387  0.986470
2026-01-24 15:10:00  1.875840 -13.392032 -1.965439  0.986475
2026-01-24 15:15:00  1.876294 -13.397841 -1.964936  0.986479


## Backtester

In [32]:
import pandas as pd
import numpy as np

def rv_switching_backtest(
    df: pd.DataFrame,
    z_col: str = "z",
    price_btc: str = "btc_close",
    price_eth: str = "eth_close",
    z_enter: float = 2.0,
    z_exit: float = 0.5,
    min_hold_bars: int = 12,          # 12 bars = 60 min on 5m
    fee_rate: float = 0.00075,        # 0.075% per side (BNB discount)
    slippage_bps: float = 0.0,        # set e.g. 1.0 for 1 bp
    initial_equity: float = 10_000.0,
):
    """
    Long-only switching RV:
      States: USDT, LONG_BTC, LONG_ETH
      Trades are executed close-to-close (your existing assumption style).
    """
    assert z_enter > z_exit >= 0

    slip = slippage_bps * 1e-4  # bps -> fraction
    eq = initial_equity

    state = "USDT"
    entry_eq = None
    entry_ts = None
    entry_price = None
    entry_asset = None
    bars_in_pos = 0

    records = []

    def exec_buy(price):
        # pay fee + slippage on buy
        return price * (1 + fee_rate + slip)

    def exec_sell(price):
        # pay fee + slippage on sell
        return price * (1 - fee_rate - slip)

    def close_position(ts, price, reason):
        nonlocal eq, state, entry_eq, entry_ts, entry_price, entry_asset, bars_in_pos
        if state == "USDT":
            return

        # equity is fully in the asset; model PnL via price ratio and sell cost
        sell_px = exec_sell(price)
        buy_px = exec_buy(entry_price)

        gross_mult = price / entry_price
        cost_mult = sell_px / price * entry_price / buy_px
        # Equivalent: net_mult = (exec_sell(price) / exec_buy(entry_price))
        net_mult = exec_sell(price) / exec_buy(entry_price)

        eq_after = entry_eq * net_mult
        ret = (eq_after / entry_eq) - 1.0

        records.append({
            "entry_ts": entry_ts,
            "exit_ts": ts,
            "asset": entry_asset,
            "entry_price": entry_price,
            "exit_price": price,
            "bars": bars_in_pos,
            "reason": reason,
            "equity_before": entry_eq,
            "equity_after": eq_after,
            "return": ret,
        })

        # reset
        eq = eq_after
        state = "USDT"
        entry_eq = entry_ts = entry_price = entry_asset = None
        bars_in_pos = 0

    def open_position(ts, asset, price):
        nonlocal eq, state, entry_eq, entry_ts, entry_price, entry_asset, bars_in_pos
        state = f"LONG_{asset}"
        entry_eq = eq
        entry_ts = ts
        entry_price = price
        entry_asset = asset
        bars_in_pos = 0

    # Iterate bars
    for ts, row in df.iterrows():
        z = row[z_col]
        btc_px = row[price_btc]
        eth_px = row[price_eth]

        # Update hold counter
        if state != "USDT":
            bars_in_pos += 1

        # Exit condition first (to allow flattening)
        if state != "USDT":
            if abs(z) <= z_exit and bars_in_pos >= 1:
                # exit current asset
                px = btc_px if state == "LONG_BTC" else eth_px
                close_position(ts, px, reason="Z_EXIT")
                continue

        # Entry / switching logic (respect min_hold to reduce churn)
        # If in position and min_hold not satisfied, do nothing.
        if state != "USDT" and bars_in_pos < min_hold_bars:
            continue

        # Desired target based on z
        target = None
        if z <= -z_enter:
            target = "ETH"
        elif z >= +z_enter:
            target = "BTC"
        else:
            target = None

        # If target is None, optionally flatten (only if not already flat)
        if target is None:
            # no action; exit already handled by z_exit band
            continue

        # If flat -> open
        if state == "USDT":
            px = eth_px if target == "ETH" else btc_px
            open_position(ts, target, px)
            continue

        # If holding the other asset -> switch (sell then buy)
        held = "BTC" if state == "LONG_BTC" else "ETH"
        if held != target:
            # close held
            px_sell = btc_px if held == "BTC" else eth_px
            close_position(ts, px_sell, reason=f"SWITCH_{held}_TO_{target}")
            # open new
            px_buy = eth_px if target == "ETH" else btc_px
            open_position(ts, target, px_buy)

    # Close any open position at the end (optional)
    if state != "USDT":
        last_ts = df.index[-1]
        last_row = df.iloc[-1]
        px = last_row[price_btc] if state == "LONG_BTC" else last_row[price_eth]
        close_position(last_ts, px, reason="EOD")

    trades = pd.DataFrame(records)
    summary = {
        "trades": len(trades),
        "total_return": (eq / initial_equity) - 1.0,
        "final_equity": eq,
        "avg_trade_return": trades["return"].mean() if len(trades) else 0.0,
        "win_rate": (trades["return"] > 0).mean() if len(trades) else 0.0,
        "median_trade_return": trades["return"].median() if len(trades) else 0.0,
    }
    return trades, summary


In [34]:
import pandas as pd
import numpy as np

def rv_switching_backtest(
    df: pd.DataFrame,
    z_col: str = "z",
    price_btc: str = "btc_close",
    price_eth: str = "eth_close",
    corr_col: str = "corr",
    eth_trend_col: str = "eth_trend_ok",
    corr_min: float = 0.95,
    z_enter: float = 2.0,
    z_exit: float = 0.5,
    min_hold_bars: int = 12,
    fee_rate: float = 0.00075,
    slippage_bps: float = 0.0,
    initial_equity: float = 10_000.0,
    exit_on_corr_break: bool = True,   # <— recommended
):
    assert z_enter > z_exit >= 0
    for c in [z_col, price_btc, price_eth, corr_col, eth_trend_col]:
        if c not in df.columns:
            raise KeyError(f"Missing required column: {c}")

    slip = slippage_bps * 1e-4
    eq = initial_equity

    state = "USDT"
    entry_eq = entry_ts = entry_price = entry_asset = None
    bars_in_pos = 0
    records = []

    def exec_buy(price):  return price * (1 + fee_rate + slip)
    def exec_sell(price): return price * (1 - fee_rate - slip)

    def close_position(ts, price, reason):
        nonlocal eq, state, entry_eq, entry_ts, entry_price, entry_asset, bars_in_pos
        if state == "USDT":
            return

        net_mult = exec_sell(price) / exec_buy(entry_price)
        eq_after = entry_eq * net_mult
        ret = (eq_after / entry_eq) - 1.0

        records.append({
            "entry_ts": entry_ts, "exit_ts": ts, "asset": entry_asset,
            "entry_price": entry_price, "exit_price": price,
            "bars": bars_in_pos, "reason": reason,
            "equity_before": entry_eq, "equity_after": eq_after,
            "return": ret,
        })

        eq = eq_after
        state = "USDT"
        entry_eq = entry_ts = entry_price = entry_asset = None
        bars_in_pos = 0

    def open_position(ts, asset, price):
        nonlocal state, entry_eq, entry_ts, entry_price, entry_asset, bars_in_pos
        state = f"LONG_{asset}"
        entry_eq = eq
        entry_ts = ts
        entry_price = price
        entry_asset = asset
        bars_in_pos = 0

    for ts, row in df.iterrows():
        z = float(row[z_col])
        btc_px = float(row[price_btc])
        eth_px = float(row[price_eth])
        corr = float(row[corr_col])
        eth_ok = bool(row[eth_trend_col])

        if state != "USDT":
            bars_in_pos += 1

        # Defensive: if corr breaks while holding, optionally exit
        if exit_on_corr_break and state != "USDT" and corr < corr_min and bars_in_pos >= 1:
            px = btc_px if state == "LONG_BTC" else eth_px
            close_position(ts, px, reason="CORR_BREAK_EXIT")
            continue

        # Normal exit (mean reversion)
        if state != "USDT" and abs(z) <= z_exit and bars_in_pos >= 1:
            px = btc_px if state == "LONG_BTC" else eth_px
            close_position(ts, px, reason="Z_EXIT")
            continue

        # Respect min_hold
        if state != "USDT" and bars_in_pos < min_hold_bars:
            continue

        # Decide target (only when structure is valid)
        target = None
        if corr >= corr_min:
            if z <= -z_enter and eth_ok:
                target = "ETH"
            elif z >= +z_enter:
                target = "BTC"

        if target is None:
            continue

        if state == "USDT":
            open_position(ts, target, eth_px if target == "ETH" else btc_px)
            continue

        held = "BTC" if state == "LONG_BTC" else "ETH"
        if held != target:
            close_position(ts, btc_px if held == "BTC" else eth_px, reason=f"SWITCH_{held}_TO_{target}")
            open_position(ts, target, eth_px if target == "ETH" else btc_px)

    if state != "USDT":
        last_ts = df.index[-1]
        last_row = df.iloc[-1]
        px = float(last_row[price_btc]) if state == "LONG_BTC" else float(last_row[price_eth])
        close_position(last_ts, px, reason="EOD")

    trades = pd.DataFrame(records)
    summary = {
        "trades": len(trades),
        "total_return": (eq / initial_equity) - 1.0,
        "final_equity": eq,
        "avg_trade_return": trades["return"].mean() if len(trades) else 0.0,
        "win_rate": (trades["return"] > 0).mean() if len(trades) else 0.0,
        "median_trade_return": trades["return"].median() if len(trades) else 0.0,
    }
    return trades, summary


In [37]:
def add_eth_trend(df, ma_bars=12*24*7):  # ~1 week MA on 5m
    out = df.copy()
    out["eth_ma"] = out["eth_close"].rolling(ma_bars).mean()
    out["eth_trend_ok"] = out["eth_close"] > out["eth_ma"]
    return out

joined3 = add_eth_trend(joined2).dropna()


## Run Backtester

In [43]:
# joined2 is your aligned df with rv features and dropna() already done
trades, summary = rv_switching_backtest(
    joined3,   # <-- IMPORTANT
    z_enter=z_enter,
    z_exit=z_exit,
    min_hold_bars=min_hold,
    fee_rate=0.00075,
    slippage_bps=0.0,
    corr_min=0.85,            # if you exposed it as a param
    exit_on_corr_break = False,
    eth_trend_col="eth_trend_ok"
)

print(summary)
print(trades["reason"].value_counts())
print(trades.tail())


{'trades': 6, 'total_return': 0.4060553687356563, 'final_equity': 14060.553687356563, 'avg_trade_return': np.float64(0.06258935117282949), 'win_rate': np.float64(0.6666666666666666), 'median_trade_return': np.float64(0.03757080097944987)}
reason
Z_EXIT    6
Name: count, dtype: int64
             entry_ts             exit_ts asset  entry_price  exit_price  \
1 2025-06-19 15:10:00 2025-06-21 06:20:00   BTC    104389.06   103513.38   
2 2025-07-09 23:15:00 2025-07-11 00:35:00   ETH      2778.63     2932.29   
3 2025-08-07 17:05:00 2025-08-11 03:15:00   ETH      3811.27     4299.71   
4 2025-11-30 11:35:00 2025-12-02 00:35:00   BTC     91152.50    86462.34   
5 2026-01-13 05:15:00 2026-01-13 20:15:00   ETH      3141.66     3213.78   

   bars  reason  equity_before  equity_after    return  
1   470  Z_EXIT   12366.793813  12244.672703 -0.009875  
2   304  Z_EXIT   12244.672703  12902.442674  0.053719  
3   986  Z_EXIT   12902.442674  14534.160270  0.126466  
4   444  Z_EXIT   14534.160270 

## Grid Run

In [35]:
z_enter_list = [1.5, 2.0, 2.5, 3.0]
z_exit_list  = [0.25, 0.5, 0.75, 1.0]
min_hold_list = [6, 12, 24, 48]


In [42]:
results = []

for z_enter in z_enter_list:
    for z_exit in z_exit_list:
        if z_exit >= z_enter:
            continue
        for min_hold in min_hold_list:
            trades, summary = rv_switching_backtest(
                joined3,   # <-- IMPORTANT
                z_enter=z_enter,
                z_exit=z_exit,
                min_hold_bars=min_hold,
                fee_rate=0.00075,
                slippage_bps=0.0,
                corr_min=0.85,            # if you exposed it as a param
                exit_on_corr_break = False,
                eth_trend_col="eth_trend_ok"
            )

            results.append({
                "z_enter": z_enter,
                "z_exit": z_exit,
                "min_hold": min_hold,
                "trades": summary["trades"],
                "total_return": summary["total_return"],
                "avg_trade_return": summary["avg_trade_return"],
                "win_rate": summary["win_rate"],
            })

grid = pd.DataFrame(results).sort_values(["total_return","avg_trade_return"], ascending=False)
grid.head(10)


,z_enter,z_exit,min_hold,trades,total_return,avg_trade_return,win_rate
48,3.0,0.25,6,6,0.419006,0.064347,0.666667
49,3.0,0.25,12,6,0.419006,0.064347,0.666667
50,3.0,0.25,24,6,0.419006,0.064347,0.666667
51,3.0,0.25,48,6,0.419006,0.064347,0.666667
52,3.0,0.50,6,6,0.409141,0.062507,0.666667
53,3.0,0.50,12,6,0.409141,0.062507,0.666667
54,3.0,0.50,24,6,0.409141,0.062507,0.666667
55,3.0,0.50,48,6,0.409141,0.062507,0.666667
60,3.0,1.00,6,6,0.406055,0.062589,0.666667
61,3.0,1.00,12,6,0.406055,0.062589,0.666667


In [46]:
best_cfg = dict(
    z_enter=3.0,
    z_exit=0.5,
    min_hold_bars=12,
    corr_min=0.85,
    exit_on_corr_break=False,
)

rv_func = lambda df, **kw: rv_switching_backtest(
    df,
    eth_trend_col="eth_trend_ok",
    **best_cfg,
    **kw
)

roll = rolling_baseline_comparison(
    joined3,
    rv_func,
    window_days=60,
    step_days=5,
    initial_equity=10_000,
    fee_rate=0.00100,    # stress again
    slippage_bps=2.0,
)

roll.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])


,start,end,btc_ret,eth_ret,rv_ret
count,57,57,57.000000,57.000000,57.000000
mean,2025-07-05 16:20:00.000000256,2025-09-03 16:20:00.000000256,0.016911,0.141304,0.074834
min,2025-02-15 16:20:00,2025-04-16 16:20:00,-0.274065,-0.413315,-0.049043
10%,2025-03-15 16:20:00,2025-05-14 16:20:00,-0.205695,-0.303702,-0.049043
25%,2025-04-26 16:20:00,2025-06-25 16:20:00,-0.100525,-0.213670,0.000000
50%,2025-07-05 16:20:00,2025-09-03 16:20:00,0.048523,0.126324,0.052718
75%,2025-09-13 16:20:00,2025-11-12 16:20:00,0.103198,0.425711,0.199190
90%,2025-10-25 16:20:00,2025-12-24 16:20:00,0.240200,0.631531,0.218488
max,2025-11-22 16:20:00,2026-01-21 16:20:00,0.321760,0.989547,0.218488
std,NaN,NaN,0.154330,0.372896,0.103056


In [47]:
{
  "RV > BTC": float((roll["rv_ret"] > roll["btc_ret"]).mean()),
  "RV > ETH": float((roll["rv_ret"] > roll["eth_ret"]).mean()),
}


{'RV > BTC': 0.631578947368421, 'RV > ETH': 0.42105263157894735}

In [40]:
top = grid.copy()

# filter out too-few-trade configs (avoid lucky results)
top = top[top["trades"] >= 15]

# pick 3 representatives
cands = pd.concat([
    top.head(1),  # best
    top.sort_values("trades").head(1),  # lowest turnover among good ones
    top.sort_values("avg_trade_return", ascending=False).head(1)  # strongest per-trade edge
]).drop_duplicates()

cands


,z_enter,z_exit,min_hold,trades,total_return,avg_trade_return,win_rate


In [15]:
def run_candidate(df, z_enter, z_exit, min_hold, fee, slip):
    _, s = rv_switching_backtest(
        df, z_enter=z_enter, z_exit=z_exit, min_hold_bars=min_hold,
        fee_rate=fee, slippage_bps=slip
    )
    return s

stress = []
for _, r in cands.iterrows():
    for fee, slip, label in [
        (0.00075, 0.0, "fee0.075_slip0"),
        (0.00075, 2.0, "fee0.075_slip2"),
        (0.00100, 2.0, "fee0.10_slip2"),
    ]:
        s = run_candidate(joined2, r.z_enter, r.z_exit, r.min_hold, fee, slip)
        stress.append({**r.to_dict(), "scenario": label, **s})

stress_df = pd.DataFrame(stress)[
    ["scenario","z_enter","z_exit","min_hold","trades","total_return","avg_trade_return","win_rate","final_equity"]
].sort_values(["scenario","total_return"], ascending=[True, False])

stress_df


,scenario,z_enter,z_exit,min_hold,trades,total_return,avg_trade_return,win_rate,final_equity
0,fee0.075_slip0,3.0,0.5,6.0,17,0.640862,0.031401,0.647059,16408.615985
1,fee0.075_slip2,3.0,0.5,6.0,17,0.629742,0.030988,0.647059,16297.415824
2,fee0.10_slip2,3.0,0.5,6.0,17,0.615947,0.030473,0.647059,16159.474710


In [17]:
best_cfg = stress_df[stress_df["scenario"]=="fee0.10_slip2"].sort_values("total_return", ascending=False).iloc[0]
best_cfg


scenario            fee0.10_slip2
z_enter                       3.0
z_exit                        0.5
min_hold                      6.0
trades                         17
total_return             0.615947
avg_trade_return         0.030473
win_rate                 0.647059
final_equity          16159.47471
Name: 2, dtype: object

In [24]:
rv_func = lambda df, **kw: rv_switching_backtest(
    df,
    z_enter=float(best_cfg["z_enter"]),
    z_exit=float(best_cfg["z_exit"]),
    min_hold_bars=int(best_cfg["min_hold"]),
    **kw
)

roll = rolling_baseline_comparison(
    joined2,
    rv_func,
    window_days=60,
    step_days=5,
    initial_equity=10_000,
    fee_rate=0.00100,     # or 0.00075
    slippage_bps=2.0,     # set what you want to stress-test
)

roll.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])


,start,end,btc_ret,eth_ret,rv_ret
count,58,58,58.000000,58.000000,58.000000
mean,2025-07-01 04:24:59.999999744,2025-08-30 04:24:59.999999744,0.016350,0.133609,0.102660
min,2025-02-08 16:25:00,2025-04-09 16:25:00,-0.288495,-0.429550,-0.117198
10%,2025-03-09 04:25:00,2025-05-08 04:25:00,-0.200344,-0.312307,-0.017920
25%,2025-04-20 22:25:00,2025-06-19 22:25:00,-0.102980,-0.196107,0.031429
50%,2025-07-01 04:25:00,2025-08-30 04:25:00,0.013188,0.129034,0.077303
75%,2025-09-10 10:25:00,2025-11-09 10:25:00,0.127509,0.429589,0.184696
90%,2025-10-23 04:25:00,2025-12-22 04:25:00,0.239030,0.661222,0.269840
max,2025-11-20 16:25:00,2026-01-19 16:25:00,0.364118,1.141367,0.309848
std,NaN,NaN,0.158936,0.382379,0.103263


In [25]:
{
  "RV > BTC": float((roll["rv_ret"] > roll["btc_ret"]).mean()),
  "RV > ETH": float((roll["rv_ret"] > roll["eth_ret"]).mean()),
}


{'RV > BTC': 0.7413793103448276, 'RV > ETH': 0.5}

## Baseline 1: Buy & Hold baseline (BTC / ETH / USDT)

In [19]:
import pandas as pd

def buy_and_hold_baseline(
    df: pd.DataFrame,
    price_col: str,
    fee_rate: float = 0.00075,
    slippage_bps: float = 0.0,
    initial_equity: float = 10_000.0,
):
    """
    Buy at first bar, sell at last bar, fee on both sides.
    """
    slip = slippage_bps * 1e-4

    entry_price = float(df[price_col].iloc[0])
    exit_price  = float(df[price_col].iloc[-1])

    buy_px  = entry_price * (1 + fee_rate + slip)
    sell_px = exit_price  * (1 - fee_rate - slip)

    net_mult = sell_px / buy_px
    final_equity = initial_equity * net_mult

    return {
        "final_equity": final_equity,
        "total_return": final_equity / initial_equity - 1.0,
        "start": df.index[0],
        "end": df.index[-1],
        "bars": len(df),
    }


## Baseline 2: Rolling-window comparison (RV vs BTC hold vs ETH hold)

In [20]:
def rolling_baseline_comparison(
    df: pd.DataFrame,
    rv_func,
    window_days: int = 60,
    step_days: int = 5,
    initial_equity: float = 10_000.0,
    fee_rate: float = 0.00075,
    slippage_bps: float = 0.0,
    min_bars: int = 100,
):
    """
    Slides a fixed window through history and compares:
      - Buy & Hold BTC
      - Buy & Hold ETH
      - RV strategy (rv_func)

    rv_func must accept (df, initial_equity=..., fee_rate=..., slippage_bps=...)
    and return (trades_df, summary_dict) where summary_dict has 'total_return'.
    """
    results = []

    window = pd.Timedelta(days=window_days)
    step = pd.Timedelta(days=step_days)

    start = df.index.min()
    end_max = df.index.max()

    while start + window <= end_max:
        end = start + window
        sub = df.loc[start:end]

        if len(sub) < min_bars:
            start += step
            continue

        btc = buy_and_hold_baseline(
            sub, "btc_close", fee_rate=fee_rate, slippage_bps=slippage_bps, initial_equity=initial_equity
        )
        eth = buy_and_hold_baseline(
            sub, "eth_close", fee_rate=fee_rate, slippage_bps=slippage_bps, initial_equity=initial_equity
        )

        _, rv = rv_func(
            sub,
            initial_equity=initial_equity,
            fee_rate=fee_rate,
            slippage_bps=slippage_bps,
        )

        results.append({
            "start": sub.index[0],
            "end": sub.index[-1],
            "btc_ret": btc["total_return"],
            "eth_ret": eth["total_return"],
            "rv_ret": float(rv["total_return"]),
        })

        start += step

    return pd.DataFrame(results)


In [23]:
initial_equity = 10_000.0
fee = 0.00075
slip = 0.0

btc_hold = buy_and_hold_baseline(joined2, "btc_close", fee_rate=fee, slippage_bps=slip, initial_equity=initial_equity)
eth_hold = buy_and_hold_baseline(joined2, "eth_close", fee_rate=fee, slippage_bps=slip, initial_equity=initial_equity)

baseline_table = pd.DataFrame([
    {"strategy": "USDT (do nothing)", "total_return": 0.0, "final_equity": initial_equity},
    {"strategy": "Buy & Hold BTC", **{k: btc_hold[k] for k in ["total_return","final_equity"]}},
    {"strategy": "Buy & Hold ETH", **{k: eth_hold[k] for k in ["total_return","final_equity"]}},
])

baseline_table


,strategy,total_return,final_equity
0,USDT (do nothing),0.000000,10000.000000
1,Buy & Hold BTC,-0.069278,9307.220699
2,Buy & Hold ETH,0.135933,11359.328358
